# Modelo
Para la construcción del modelo se utilizó `tensorflow-gpu`, pues aumenta muchísimo la velocidad de procesamiento en éste tipo de cargas.

In [1]:
import tensorflow as tf
import numpy as np
import tensorflow.keras.layers as tfl

print(tf.config.list_physical_devices('GPU'))

2025-07-23 23:24:43.057666: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1753331083.069348    5840 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1753331083.072993    5840 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-07-23 23:24:43.086157: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


Se cargan los _datasets_ y se separan en sus versiones de entrenamiento (`_train`) y de prueba (`_test`) además de combinar los datos de entrada en una sola _lista_ para mayor comodidad.

In [2]:
HU = np.load('hu_X.npy')
PA = np.load('pa_X.npy')
VX = np.load('vx_X.npy')
VY = np.load('vy_X.npy')
TP = np.load('tp_X.npy')
PR = np.load('pr_X.npy')
Y = np.load('Y.npy')

In [3]:
HU_train = HU[:4499,:,:,:]
HU_test = HU[4500:,:,:,:]
PA_train = PA[:4499,:,:,:]
PA_test = PA[4500:,:,:,:]
VX_train = VX[:4499,:,:,:]
VX_test = VX[4500:,:,:,:]
VY_train = VY[:4499,:,:,:]
VY_test = VY[4500:,:,:,:]
TP_train = TP[:4499,:,:,:]
TP_test = TP[4500:,:,:,:]
PR_train = PR[:4499,:,:,:]
PR_test = PR[4500:,:,:,:]
Y_train = Y[:4499,:,:]
Y_test = Y[4500:,:,:]

X_train = [HU_train, PA_train, VX_train, VY_train, TP_train, PR_train]
X_test = [HU_test, PA_test, VX_test, VY_test, TP_test, PR_test]

## Arquitectura
En ésta celda se define la arquitectura del modelo, como se puede ver toma 6 entradas de forma `(14, 24, 6)`, y esas dimensiones significan el número de dias, horas y estación. Las entradas tienen ésta forma pues la idea del modelo es poder predecir el clima del día siguiente basandose en el clima de las últimas dos semanas.
Despues se procesa cada entrada individualmente por dos capas convolucionales para identificar los _features_ de cada _dataset_, posteriormente se concatenan y se tratan como un solo volumen que pasa por 5 convoluciones mas que aumentan la cantidad de filtros, despues se aplana y se pasa a una red completamente conectada de 4 capas y finalmente pasa por otras 4 capas convolucionales despues de ser reformada para generar una salida con la forma deseada, `(24, 6, 6)`, que indican las horas en un dia, estaciones y métricas en un solo día. Justo lo que estabamos buscando.

In [4]:
#MODEL
hu_in = tf.keras.Input(shape=(14,24,6))
pa_in = tf.keras.Input(shape=(14,24,6))
vx_in = tf.keras.Input(shape=(14,24,6))
vy_in = tf.keras.Input(shape=(14,24,6))
tp_in = tf.keras.Input(shape=(14,24,6))
pr_in = tf.keras.Input(shape=(14,24,6))


hu = tfl.Conv2D(filters=16, kernel_size=7, strides=(5, 5), padding='same', activation='relu')(hu_in)
pa = tfl.Conv2D(filters=16, kernel_size=7, strides=(5, 5), padding='same', activation='relu')(pa_in)
vx = tfl.Conv2D(filters=16, kernel_size=7, strides=(5, 5), padding='same', activation='relu')(vx_in)
vy = tfl.Conv2D(filters=16, kernel_size=7, strides=(5, 5), padding='same', activation='relu')(vy_in)
tp = tfl.Conv2D(filters=16, kernel_size=7, strides=(5, 5), padding='same', activation='relu')(tp_in)
pr = tfl.Conv2D(filters=16, kernel_size=7, strides=(5, 5), padding='same', activation='relu')(pr_in)

hu = tfl.BatchNormalization()(hu)
pa = tfl.BatchNormalization()(pa)
vx = tfl.BatchNormalization()(vx)
vy = tfl.BatchNormalization()(vy)
tp = tfl.BatchNormalization()(tp)
pr = tfl.BatchNormalization()(pr)

hu = tfl.Conv2D(filters=32, kernel_size=7, strides=(5, 5), padding='same', activation='sigmoid')(hu)
pa = tfl.Conv2D(filters=32, kernel_size=7, strides=(5, 5), padding='same', activation='sigmoid')(pa)
vx = tfl.Conv2D(filters=32, kernel_size=7, strides=(5, 5), padding='same', activation='sigmoid')(vx)
vy = tfl.Conv2D(filters=32, kernel_size=7, strides=(5, 5), padding='same', activation='sigmoid')(vy)
tp = tfl.Conv2D(filters=32, kernel_size=7, strides=(5, 5), padding='same', activation='sigmoid')(tp)
pr = tfl.Conv2D(filters=32, kernel_size=7, strides=(5, 5), padding='same', activation='sigmoid')(pr)

cache = tfl.Concatenate()([hu, pa, vx, vy, tp, pr])
cache = tfl.Conv2D(filters=32, kernel_size=5, strides=(3, 3), padding='same', activation='relu')(cache)
cache = tfl.Conv2D(filters=32, kernel_size=5, strides=(3, 3), padding='same', activation='sigmoid')(cache)
cache = tfl.Conv2D(filters=64, kernel_size=3, strides=(1, 1), padding='same', activation='relu')(cache)
cache = tfl.Conv2D(filters=64, kernel_size=3, strides=(1, 1), padding='same', activation='sigmoid')(cache)
cache = tfl.Conv2D(filters=64, kernel_size=3, strides=(1, 1), padding='same', activation='sigmoid')(cache)
cache = tfl.Flatten()(cache)
cache = tfl.Dense(units=16, activation='relu')(cache)
cache = tfl.Dense(units=32, activation='sigmoid')(cache)
cache = tfl.Dense(units=64, activation='relu')(cache)
cache = tfl.Dense(units=64, activation='sigmoid')(cache)
cache = tfl.Reshape((8,4,2))(cache)
cache = tfl.ZeroPadding2D(padding=2)(cache)
cache = tfl.Conv2D(filters=4, kernel_size=3, strides=(1,1), padding='valid', activation='relu')(cache)
cache = tfl.ZeroPadding2D(padding=5)(cache)
cache = tfl.Conv2D(filters=8, kernel_size=5, strides=(1,1), padding='valid', activation='sigmoid')(cache)
cache = tfl.ZeroPadding2D(padding=3)(cache)
cache = tfl.Conv2D(filters=12, kernel_size=3, strides=(1,3), padding='valid', activation='sigmoid')(cache)
cache = tfl.ZeroPadding2D(padding=(3,1))(cache)
output = tfl.Conv2D(filters=6, kernel_size=3, strides=(1,1), padding='valid', activation='sigmoid')(cache)

model = tf.keras.models.Model(inputs=[hu_in, pa_in, vx_in, vy_in, tp_in, pr_in], outputs=output)

I0000 00:00:1753331090.612322    5840 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 3548 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 2060, pci bus id: 0000:0b:00.0, compute capability: 7.5


In [5]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 14, 24, 6) │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_1       │ (None, 14, 24, 6) │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_2       │ (None, 14, 24, 6) │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_3       │ (None, 14, 24, 6) │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_4       │ (None, 14, 24, 6) │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_5       │ (None, 14, 24, 6) │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d (Conv2D)     │ (None, 3, 5, 16)  │      4,720 │ input_layer[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_1 (Conv2D)   │ (None, 3, 5, 16)  │      4,720 │ input_layer_1[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_2 (Conv2D)   │ (None, 3, 5, 16)  │      4,720 │ input_layer_2[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_3 (Conv2D)   │ (None, 3, 5, 16)  │      4,720 │ input_layer_3[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_4 (Conv2D)   │ (None, 3, 5, 16)  │      4,720 │ input_layer_4[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_5 (Conv2D)   │ (None, 3, 5, 16)  │      4,720 │ input_layer_5[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalization │ (None, 3, 5, 16)  │         64 │ conv2d[0][0]      │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 3, 5, 16)  │         64 │ conv2d_1[0][0]    │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 3, 5, 16)  │         64 │ conv2d_2[0][0]    │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 3, 5, 16)  │         64 │ conv2d_3[0][0]    │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 3, 5, 16)  │         64 │ conv2d_4[0][0]    │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 3, 5, 16)  │         64 │ conv2d_5[0][0]    │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_6 (Conv2D)   │ (None, 1, 1, 32)  │     25,120 │ batch_normalizat

 Total params: 461,310 (1.76 MB)

 Trainable params: 461,118 (1.76 MB)

 Non-trainable params: 192 (768.00 B)

Finalmente se puede entrenar el modelo, se entrenó con 1000 épocas, pues el dataset es bastante pequeño y se corría un gran riesgo de sobreentrenar dicho modelo, aún asi parece que fue sobreentrenado dados los valores finales de _accuracy_, _val-accuracy_, _loss_ y _val-loss_.

In [7]:
model.fit(X_train, Y_train, validation_data=(X_test, Y_test), epochs=1000)

Epoch 1/1000


I0000 00:00:1753331155.981024    5881 service.cc:148] XLA service 0x7f4534009620 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1753331155.981053    5881 service.cc:156]   StreamExecutor device (0): NVIDIA GeForce RTX 2060, Compute Capability 7.5
2025-07-23 23:25:56.110274: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1753331156.701186    5881 cuda_dnn.cc:529] Loaded cuDNN version 90101
2025-07-23 23:25:57.416497: I external/local_xla/xla/service/gpu/autotuning/conv_algorithm_picker.cc:557] Omitted potentially buggy algorithm eng14{} for conv (f32[32,4,10,6]{3,2,1,0}, u8[0]{0}) custom-call(f32[32,2,8,4]{3,2,1,0}, f32[4,2,3,3]{3,2,1,0}, f32[4]{0}), window={size=3x3 pad=2_2x2_2}, dim_labels=bf01_oi01->bf01, custom_call_target="__cudnn$convBiasActivationForward", backend_config={"cudnn_conv_backend_config":{"activa

 38/141 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.0468 - loss: 1500.5160    

I0000 00:00:1753331161.170392    5881 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


135/141 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.0673 - loss: 1325.1633

2025-07-23 23:26:02.623209: I external/local_xla/xla/service/gpu/autotuning/conv_algorithm_picker.cc:557] Omitted potentially buggy algorithm eng14{} for conv (f32[19,4,10,6]{3,2,1,0}, u8[0]{0}) custom-call(f32[19,2,8,4]{3,2,1,0}, f32[4,2,3,3]{3,2,1,0}, f32[4]{0}), window={size=3x3 pad=2_2x2_2}, dim_labels=bf01_oi01->bf01, custom_call_target="__cudnn$convBiasActivationForward", backend_config={"cudnn_conv_backend_config":{"activation_mode":"kNone","conv_result_scale":1,"leakyrelu_alpha":0,"side_input_scale":0},"force_earliest_schedule":false,"operation_queue_id":"0","wait_on_operation_queues":[]}
2025-07-23 23:26:02.714913: I external/local_xla/xla/service/gpu/autotuning/conv_algorithm_picker.cc:557] Omitted potentially buggy algorithm eng14{} for conv (f32[19,6,24,6]{3,2,1,0}, u8[0]{0}) custom-call(f32[19,12,20,6]{3,2,1,0}, f32[6,12,3,3]{3,2,1,0}, f32[6]{0}), window={size=3x3 pad=3_3x1_1}, dim_labels=bf01_oi01->bf01, custom_call_target="__cudnn$convBiasActivationForward", backend_conf

141/141 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step - accuracy: 0.0677 - loss: 1321.3636

2025-07-23 23:26:07.130028: I external/local_xla/xla/service/gpu/autotuning/conv_algorithm_picker.cc:557] Omitted potentially buggy algorithm eng14{} for conv (f32[32,4,10,6]{3,2,1,0}, u8[0]{0}) custom-call(f32[32,2,8,4]{3,2,1,0}, f32[4,2,3,3]{3,2,1,0}, f32[4]{0}), window={size=3x3 pad=2_2x2_2}, dim_labels=bf01_oi01->bf01, custom_call_target="__cudnn$convBiasActivationForward", backend_config={"cudnn_conv_backend_config":{"activation_mode":"kRelu","conv_result_scale":1,"leakyrelu_alpha":0,"side_input_scale":0},"force_earliest_schedule":false,"operation_queue_id":"0","wait_on_operation_queues":[]}
2025-07-23 23:26:08.027057: I external/local_xla/xla/service/gpu/autotuning/conv_algorithm_picker.cc:557] Omitted potentially buggy algorithm eng14{} for conv (f32[8,4,10,6]{3,2,1,0}, u8[0]{0}) custom-call(f32[8,2,8,4]{3,2,1,0}, f32[4,2,3,3]{3,2,1,0}, f32[4]{0}), window={size=3x3 pad=2_2x2_2}, dim_labels=bf01_oi01->bf01, custom_call_target="__cudnn$convBiasActivationForward", backend_config={"

141/141 ━━━━━━━━━━━━━━━━━━━━ 16s 53ms/step - accuracy: 0.0678 - loss: 1320.7738 - val_accuracy: 0.0833 - val_loss: 1369.7798
Epoch 2/1000
141/141 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.0833 - loss: 1390.8744 - val_accuracy: 0.0833 - val_loss: 1457.0887
Epoch 3/1000
141/141 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.0834 - loss: 1417.1752 - val_accuracy: 0.0833 - val_loss: 1414.0396
Epoch 4/1000
141/141 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.0838 - loss: 1379.4031 - val_accuracy: 0.0833 - val_loss: 1391.2734
Epoch 5/1000
141/141 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.0849 - loss: 1350.4191 - val_accuracy: 0.0833 - val_loss: 1368.4086
Epoch 6/1000
141/141 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.0885 - loss: 1318.3499 - val_accuracy: 0.0833 - val_loss: 1352.7020
Epoch 7/1000
141/141 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.0833 - loss: 1287.2617 - val_accuracy: 0.0833 - val_loss: 1339.3934
Epoch 8/1000
141/141 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - acc